<a href="https://www.kaggle.com/code/sharabhojha/modified-chord-generation-lstm?scriptVersionId=234014082" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import re
import pandas as pd
###!pip install liac-arff

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Preprocessing steps:
# 1: condense onset files to just onset times and onset notes
"""
import re
import pandas as pd
import arff

# Directory path to your annotations folder
annotations_dir = "/kaggle/input/aam-annotations/AAM-annotations/"
num = 0

# Iterate through all files in the directory
for filename in os.listdir(annotations_dir):
    if "onsets" in filename and filename.endswith(".arff"):  # Ensure it's an ARFF file with 'onsets' in its name
        num += 1
        file_path = os.path.join(annotations_dir, filename)
        
        # Read the ARFF file
        def read_arff(file_path):
            with open(file_path, 'r') as f:
                arff_data = arff.load(f)
            return pd.DataFrame(arff_data['data'], columns=[attr[0] for attr in arff_data['attributes']])
        
        # Use this function instead of manual parsing
        df = read_arff(file_path)

        # Convert numeric columns where possible
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col])  # Convert if possible
            except ValueError:
                pass  # Keep as string if conversion fails

        all_onsets = []

        # Vectorized operation to collect all onset events
        df.drop(columns=["Onset events of Drums"], inplace=True)
        onset_columns = df.columns[df.apply(lambda col: col.astype(str).str.contains(r"\[", regex=True)).any()]
        df["Onset events"] = df[onset_columns].apply(lambda row: [int(x) for x in re.findall(r'\d+', ''.join(row.astype(str)))], axis=1)
        df.drop(onset_columns, axis=1, inplace=True)

        # Save the processed DataFrame to a new CSV file
        output_file = re.search("(\d+)", filename).group(0) + "_onset_condensed.csv"
        df.to_csv(output_file, index=False)

        if num % 100 == 0:
            print(f"Processed {filename} and saved to {output_file}")"""

'\nimport re\nimport pandas as pd\nimport arff\n\n# Directory path to your annotations folder\nannotations_dir = "/kaggle/input/aam-annotations/AAM-annotations/"\nnum = 0\n\n# Iterate through all files in the directory\nfor filename in os.listdir(annotations_dir):\n    if "onsets" in filename and filename.endswith(".arff"):  # Ensure it\'s an ARFF file with \'onsets\' in its name\n        num += 1\n        file_path = os.path.join(annotations_dir, filename)\n        \n        # Read the ARFF file\n        def read_arff(file_path):\n            with open(file_path, \'r\') as f:\n                arff_data = arff.load(f)\n            return pd.DataFrame(arff_data[\'data\'], columns=[attr[0] for attr in arff_data[\'attributes\']])\n        \n        # Use this function instead of manual parsing\n        df = read_arff(file_path)\n\n        # Convert numeric columns where possible\n        for col in df.columns:\n            try:\n                df[col] = pd.to_numeric(df[col])  # Convert 

In [3]:
# 2: encode chord names and replace said chord names with encodings in beatinfo files

chord_encodings = {0: 'A#maj', 1: 'A#min', 2: 'Amaj', 3: 'Amin', 4: 'Bmaj', 5: 'Bmin', 6: 'C#maj', 7: 'C#min', 
                   8: 'Cmaj', 9: 'Cmin', 10: 'D#maj', 11: 'D#min', 12: 'Dmaj', 13: 'Dmin', 14: 'Emaj', 15: 'Emin', 
                   16: 'F#maj', 17: 'F#min', 18: 'Fmaj', 19: 'Fmin', 20: 'G#maj', 21: 'G#min', 22: 'Gmaj', 
                   23: 'Gmin', 24: 'N.C.'}

inverted_encodings = dict(zip(chord_encodings.values(), chord_encodings.keys()))

chords = {"Cmaj": [60, 64, 67, 72], "Cmin": [60, 63, 67, 72], "C#maj": [61, 65, 68, 73], "C#min": [61, 64, 68, 73],
          "Dmaj": [62, 66, 69, 74], "Dmin": [62, 65, 69, 74], "D#maj": [63, 67, 70, 75], "D#min": [63, 66, 70, 75],
          "Emaj": [64, 68, 71, 76], "Emin": [64, 67, 71, 76], "Fmaj": [65, 69, 72, 77], "Fmin": [65, 68, 72, 77],
          "F#maj": [66, 70, 73, 78], "F#min": [66, 69, 73, 78], "Gmaj": [67, 71, 74, 79], "Gmin": [67, 70, 74, 79],
          "G#maj": [68, 72, 75, 80], "G#min": [68, 71, 75, 80], "Amaj": [69, 73, 76, 81], "Amin": [69, 72, 76, 81],
          "A#maj": [70, 74, 77, 82], "A#min": [70, 73, 77, 82], "Bmaj": [71, 75, 78, 83], "Bmin": [71, 74, 78, 83]}

midi_chord_dict = {}

for label, chord_name in chord_encodings.items():
    midi_chord_dict[label] = chords.get(chord_name, [])

midi_chord_dict.pop(24)

print(midi_chord_dict)

"""
# Directory path to your annotations folder
headers = ['Start time in seconds', 'Bar count', 'Quarter count', 'Chord name']

# Iterate through all files in the directory
for filename in os.listdir(annotations_dir):
    if "beatinfo" in filename and filename.endswith(".arff"):  # Ensure it's an ARFF file with 'beatinfo' in its name
        file_path = os.path.join(annotations_dir, filename)
        df = pd.read_csv(file_path, comment='@', header=None)
        df.columns = headers

        for i in range(df.index.size):
            df.iat[i, 3] = df.iat[i, 3].replace("'", "")
            if df.iat[i, 3] == "BASS_NOTE_EXCEPTION":
                df.iat[i, 3] = "N.C."
            df.iat[i, 3] = inverted_encodings[df.iat[i, 3]]
    
        df.to_csv(filename.replace('arff', 'csv'), index=False)"""

{0: [70, 74, 77, 82], 1: [70, 73, 77, 82], 2: [69, 73, 76, 81], 3: [69, 72, 76, 81], 4: [71, 75, 78, 83], 5: [71, 74, 78, 83], 6: [61, 65, 68, 73], 7: [61, 64, 68, 73], 8: [60, 64, 67, 72], 9: [60, 63, 67, 72], 10: [63, 67, 70, 75], 11: [63, 66, 70, 75], 12: [62, 66, 69, 74], 13: [62, 65, 69, 74], 14: [64, 68, 71, 76], 15: [64, 67, 71, 76], 16: [66, 70, 73, 78], 17: [66, 69, 73, 78], 18: [65, 69, 72, 77], 19: [65, 68, 72, 77], 20: [68, 72, 75, 80], 21: [68, 71, 75, 80], 22: [67, 71, 74, 79], 23: [67, 70, 74, 79]}


'\n# Directory path to your annotations folder\nheaders = [\'Start time in seconds\', \'Bar count\', \'Quarter count\', \'Chord name\']\n\n# Iterate through all files in the directory\nfor filename in os.listdir(annotations_dir):\n    if "beatinfo" in filename and filename.endswith(".arff"):  # Ensure it\'s an ARFF file with \'beatinfo\' in its name\n        file_path = os.path.join(annotations_dir, filename)\n        df = pd.read_csv(file_path, comment=\'@\', header=None)\n        df.columns = headers\n\n        for i in range(df.index.size):\n            df.iat[i, 3] = df.iat[i, 3].replace("\'", "")\n            if df.iat[i, 3] == "BASS_NOTE_EXCEPTION":\n                df.iat[i, 3] = "N.C."\n            df.iat[i, 3] = inverted_encodings[df.iat[i, 3]]\n    \n        df.to_csv(filename.replace(\'arff\', \'csv\'), index=False)'

In [4]:
# visualize the files

###working_dir = "/kaggle/working/"
working_dir = "/kaggle/input/aam-paired-chord-onset-dataset/"
onsets = pd.read_csv(working_dir + "0001_onset_condensed.csv")
print(onsets.head())
beatinfo = pd.read_csv(working_dir + "0001_beatinfo.csv")
print(beatinfo.head())

def align_onsets_with_chords(onsets, beatinfo):
    aligned_data = []
    for _, onset_row in onsets.iterrows():
        onset_time = onset_row['Onset time in seconds']
        # Find the chord corresponding to this onset time
        chord_row = beatinfo[beatinfo['Start time in seconds'] <= onset_time].iloc[-1]
        onset_list = eval(onset_row['Onset events'])
        if chord_row['Chord name'] != 24 and len(onset_list) > 0:
            if len(aligned_data) > 0:
                aligned_data.append((onset_list, aligned_data[-1][2], int(chord_row['Chord name'])))
                continue
            # replace second param with int(chord_row['Chord name'])
            aligned_data.append((onset_list, 24, int(chord_row['Chord name'])))
    return aligned_data

   Onset time in seconds  Onset events
0               0.000000  [41, 60, 65]
1               0.326086      [41, 60]
2               0.652173  [41, 65, 65]
3               0.978259  [41, 65, 69]
4               1.304346  [41, 65, 65]
   Start time in seconds  Bar count  Quarter count  Chord name
0               0.000000          1              1          18
1               0.652174          1              2          18
2               1.304348          1              3          18
3               1.956522          1              4          18
4               2.608696          2              1           0


In [5]:
# create aligned data for every onset and beatinfo file

all_data = []
i = 0

while i < 1000:
    i += 1
    onset_path = os.path.join(working_dir, f"{i :04d}_onset_condensed.csv")
    beatinfo_path = os.path.join(working_dir, f"{i :04d}_beatinfo.csv")
    onsets = pd.read_csv(onset_path)
    beatinfo = pd.read_csv(beatinfo_path)
    all_data.append(align_onsets_with_chords(onsets, beatinfo))
    if i % 50 == 0:
        print(i)

print(all_data[0:2])
#print(all_data[0:20])

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
[[([41, 60, 65], 24, 18), ([41, 60], 18, 18), ([41, 65, 65], 18, 18), ([41, 65, 69], 18, 18), ([41, 65, 65], 18, 18), ([41, 65, 69], 18, 18), ([41, 64, 60], 18, 18), ([41, 64, 65], 18, 18), ([46, 67, 70], 18, 18), ([46, 67, 65], 18, 0), ([46, 67, 70], 0, 0), ([46, 69, 62], 0, 0), ([46, 60, 70], 0, 0), ([46, 62], 0, 0), ([46, 65, 65], 0, 0), ([46, 65, 70], 0, 0), ([45, 69, 69], 0, 0), ([45, 64, 64], 0, 3), ([45, 65, 69], 3, 3), ([45, 69, 60], 3, 3), ([38, 69, 62], 3, 3), ([38, 65, 69], 3, 13), ([38, 60, 62], 13, 13), ([38, 65], 13, 13), ([38, 60, 62], 13, 13), ([38, 60, 69], 13, 13), ([38, 65, 62], 13, 13), ([38, 65, 65], 13, 13), ([38, 65, 62], 13, 13), ([38, 65, 65], 13, 13), ([38, 64, 69], 13, 13), ([38, 62], 13, 13), ([36, 67, 60], 13, 13), ([36, 67], 13, 8), ([36, 67, 60], 8, 8), ([36, 69, 64], 8, 8), ([36, 69, 60], 8, 8), ([36, 65, 64], 8, 8), ([36, 62, 67], 8, 8), ([36, 62, 60], 8, 8), ([41, 65, 65], 

In [6]:
# --- Setup ---
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Masking, Input, Embedding, Concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# --- Constants ---
NUM_MIDI_NOTES = 128
NUM_CHORDS = 24
CHORD_INPUT_DIM = NUM_CHORDS + 1  # +1 for 'None'
INPUT_DIM = NUM_MIDI_NOTES + CHORD_INPUT_DIM

# --- Utility Functions ---
def notes_to_vector(notes):
    vec = np.zeros(NUM_MIDI_NOTES)
    for n in notes:
        if 0 <= n < NUM_MIDI_NOTES:
            vec[n] = 1
    return vec

def build_input_sequence(sequences):
    note_seqs, chord_seqs, y_seqs = [], [], []
    for seq in sequences:
        note_seq, chord_seq, y_seq = [], [], []
        for notes, prev_chord, current_chord in seq:
            note_vec = notes_to_vector(notes)
            chord_idx = prev_chord if prev_chord is not None else NUM_CHORDS
            note_seq.append(note_vec)
            chord_seq.append(chord_idx)
            y_seq.append(current_chord)
        note_seqs.append(note_seq)
        chord_seqs.append(chord_seq)
        y_seqs.append(y_seq)

    note_padded = pad_sequences(note_seqs, padding='post', dtype='float32')
    chord_padded = pad_sequences(chord_seqs, padding='post', dtype='int32')
    y_padded = pad_sequences(y_seqs, padding='post', dtype='int32')
    return note_padded, chord_padded, y_padded

def notes_to_pitch_class_vector(notes):
    vec = np.zeros(12)
    for note in notes:
        vec[note % 12] += 1
    return vec

def normalize_pitch_class_set(notes):
    return tuple(sorted(note % 12 for note in notes))

def best_chord_match(notes, chord_templates, verbose=False):
    input_set = set(notes)
    input_pc_set = normalize_pitch_class_set(notes)
    formatted_notes = notes_to_pitch_class_vector(notes).reshape(-1, 1)

    best_label = 24  # default label if no match
    best_score = -1
    best_overlap = -1

    """# First pass: check for exact pitch class match
    for label, chord_notes in chord_templates.items():
        if normalize_pitch_class_set(chord_notes) == input_pc_set:
            if verbose:
                print(f"Exact match found: {label}")
            return label"""

    # Second pass: compute cosine similarity for candidates with any overlap
    for chord_label, chord_notes in chord_templates.items():
        template_set = set(chord_notes)
        if len(input_set & template_set) == 0:
            continue  # skip chords with no shared notes

        formatted_chord = notes_to_pitch_class_vector(chord_notes).reshape(-1, 1)
        curr_score = cosine_similarity(formatted_notes, formatted_chord)[0][0]
        overlap_size = len(input_set & template_set)

        if verbose:
            print(f"Comparing with {chord_label}: cosine={curr_score:.3f}, overlap={overlap_size}")

        if (curr_score > best_score or
            (curr_score == best_score and overlap_size > best_overlap)):
            best_score = curr_score
            best_overlap = overlap_size
            best_label = chord_label

    return best_label
    
# --- Train/Test Split ---
train_data, test_data = train_test_split(all_data, test_size=0.2) # random_state=42

# --- Vectorize ---
note_train, chord_train, y_train = build_input_sequence(train_data)
note_test, chord_test, y_test = build_input_sequence(test_data)

# --- Model ---
note_input = Input(shape=(None, NUM_MIDI_NOTES), name='notes')
chord_input = Input(shape=(None,), dtype='int32', name='prev_chords')

# Embed previous chord (25 possible values → 16-dim vectors)
chord_embedding = Embedding(input_dim=CHORD_INPUT_DIM, output_dim=16)(chord_input)

# Combine note vector and chord embedding
combined_input = Concatenate()([note_input, chord_embedding])

x = Masking(mask_value=0.0)(combined_input)
x = LSTM(128, return_sequences=True)(x)
output = Dense(NUM_CHORDS, activation='softmax')(x)

model = Model(inputs=[note_input, chord_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# --- Train ---
model.fit(
    [note_train, chord_train],
    y_train,
    epochs=10,
    batch_size=8,
    validation_split=0.2
)

# --- Inference Function ---
def predict_chords(model, note_sequence):
    note_inputs = []
    chord_inputs = []
    predictions = []
    confidences = []
    prev_chord = NUM_CHORDS  # 'None'

    for notes in note_sequence:
        note_vec = notes_to_vector(notes)
        note_inputs.append(note_vec)
        chord_inputs.append(prev_chord)

        X_notes = np.array(note_inputs)[np.newaxis, :, :]
        X_chords = np.array(chord_inputs)[np.newaxis, :]
        pred = model.predict([X_notes, X_chords], verbose=0)[0, -1]
        prev_chord = np.argmax(pred)
        confidence = float(np.max(pred))
        if confidence < 0.40:
            prev_chord = best_chord_match(notes, midi_chord_dict)
        predictions.append(prev_chord)
        confidences.append(confidence)

    return predictions, confidences

# --- Test on One Example ---
example = test_data[0]
note_sequence = [t[0] for t in example]
true_chords = [t[2] for t in example]

predicted_chords, confidences = predict_chords(model, note_sequence)

print("True Chords:     ", true_chords)
print("Predicted Chords:", predicted_chords)
print("Confidences: ", confidences)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ prev_chords (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ notes (InputLayer)        │ (None, None, 128)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 16)       │            400 │ prev_chords[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, None, 144)      │              0 │ notes[0][0],           │
│                           │                        │                │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None, 144)      │              0 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ masking (Masking)         │ (None, None, 144)      │              0 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ any (Any)                 │ (None, None)           │              0 │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, None, 128)      │        139,776 │ masking[0][0],         │
│                           │                        │                │ any[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 24)       │          3,096 │ lstm[0][0]             │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 143,272 (559.66 KB)

 Trainable params: 143,272 (559.66 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.6528 - loss: 1.8084 - val_accuracy: 0.8620 - val_loss: 0.5976
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.8902 - loss: 0.4655 - val_accuracy: 0.9510 - val_loss: 0.1811
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9583 - loss: 0.1616 - val_accuracy: 0.9779 - val_loss: 0.0988
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9784 - loss: 0.0974 - val_accuracy: 0.9877 - val_loss: 0.0692
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9857 - loss: 0.0725 - val_accuracy: 0.9906 - val_loss: 0.0536
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.9889 - loss: 0.0587 - val_accuracy: 0.9914 - val_loss: 0.0457
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9891 - loss: 0.0534 - val_accuracy: 0.9924 - val_loss: 0.0402
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9909 - loss: 0.0452 - val_accuracy: 0.9928 - v

In [7]:
for name, chord in chords.items():
    pred_chord, conf = predict_chords(model, [chord])
    print(f"{name}, {chord_encodings[pred_chord[0]]}, {conf}")

'''for name, chord in chords.items():
    pred_chord = best_chord_match(chord, midi_chord_dict)
    print(f"{name}, {chord_encodings[pred_chord]}")'''

Cmaj, Cmaj, [0.13331948220729828]
Cmin, Cmin, [0.11815690249204636]
C#maj, C#maj, [0.24101538956165314]
C#min, C#min, [0.14509643614292145]
Dmaj, Dmaj, [0.17686399817466736]
Dmin, Dmin, [0.1446687877178192]
D#maj, D#maj, [0.16283133625984192]
D#min, D#min, [0.13204045593738556]
Emaj, Emaj, [0.1571585088968277]
Emin, Emin, [0.0978943333029747]
Fmaj, Fmaj, [0.16566616296768188]
Fmin, Fmin, [0.11311611533164978]
F#maj, F#maj, [0.17769040167331696]
F#min, F#min, [0.12576565146446228]
Gmaj, Gmaj, [0.09151533246040344]
Gmin, Gmin, [0.11302464455366135]
G#maj, G#maj, [0.07778660207986832]
G#min, G#min, [0.09653036296367645]
Amaj, Amaj, [0.07700403034687042]
Amin, Amin, [0.10019289702177048]
A#maj, A#maj, [0.1319868266582489]
A#min, A#min, [0.1178470328450203]
Bmaj, Bmaj, [0.07273509353399277]
Bmin, Bmin, [0.08100653439760208]


'for name, chord in chords.items():\n    pred_chord = best_chord_match(chord, midi_chord_dict)\n    print(f"{name}, {chord_encodings[pred_chord]}")'

In [8]:
'''all_true = []
all_pred = []

for example in test_data:
    note_sequence = [t[0] for t in example]
    true_chords = [t[2] for t in example]
    predicted_chords = predict_chords(model, note_sequence)
    
    all_true.extend(true_chords)
    all_pred.extend(predicted_chords)

# Now use all_true and all_pred to make the confusion matrix
cm = confusion_matrix(all_true, all_pred, labels=range(NUM_CHORDS))

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[str(i) for i in range(NUM_CHORDS)])
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix - All Test Data")
plt.xlabel("Predicted Chord")
plt.ylabel("True Chord")
plt.show()'''

'all_true = []\nall_pred = []\n\nfor example in test_data:\n    note_sequence = [t[0] for t in example]\n    true_chords = [t[2] for t in example]\n    predicted_chords = predict_chords(model, note_sequence)\n    \n    all_true.extend(true_chords)\n    all_pred.extend(predicted_chords)\n\n# Now use all_true and all_pred to make the confusion matrix\ncm = confusion_matrix(all_true, all_pred, labels=range(NUM_CHORDS))\n\ndisp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[str(i) for i in range(NUM_CHORDS)])\nfig, ax = plt.subplots(figsize=(10, 10))\ndisp.plot(ax=ax, cmap=\'Blues\', xticks_rotation=45)\nplt.title("Confusion Matrix - All Test Data")\nplt.xlabel("Predicted Chord")\nplt.ylabel("True Chord")\nplt.show()'

In [9]:
model.save('BestChordPredictor.keras')